# set up **pyspark**

In [1]:
!python --version

Python 3.7.13


In [2]:
!pwd

/content


In [3]:
#!apt-get update -y

In [4]:
!apt-get install openjdk-8-jdk-headless -qq>/dev/null

In [5]:
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

In [6]:
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

# set Java and Hadoop

In [7]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [8]:
!pip install -q findspark
import findspark
findspark.init()

In [9]:
import os       #importing os to set environment variable
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null     
   #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64" 
   #set environment variable
  !java -version      
   #check java version
install_java()

openjdk version "11.0.16" 2022-07-19
OpenJDK Runtime Environment (build 11.0.16+8-post-Ubuntu-0ubuntu118.04)
OpenJDK 64-Bit Server VM (build 11.0.16+8-post-Ubuntu-0ubuntu118.04, mixed mode, sharing)


In [10]:
# Spark SQL
!pip install pyspark[sql]
# pandas API on Spark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
from pyspark.sql import SparkSession
spark = (SparkSession
 .builder
 .appName("PySpark 3.2.1 Setup on Google Colab")
 .getOrCreate())

In [12]:
!pip install pyspark[pandas_on_spark] plotly 
# to plot your data, you can install plotly together.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
!pip install gdown

# to upgrade
#!pip install --upgrade gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Download file from Drive

In [14]:
import gdown
# a file
url = "https://drive.google.com/drive/folders/1QgWPHV_l25Ui9L7et8mkZohAOG59UTkQ"
gdown.download_folder(url, quiet=True, use_cookies=False)

['/content/chess/chess_schema.png',
 '/content/chess/chess_wc_history_game_info.csv',
 '/content/chess/chess_wc_history_moves.csv',
 '/content/chess/eco_codes.csv']

In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import mean,col,split, col,regexp_extract, when ,lit

In [16]:
chess_games= spark.read.load("/content/chess/chess_wc_history_game_info.csv",format="csv", sep=",", inferSchema="true", header="true")
chess_moves= spark.read.load('/content/chess/chess_wc_history_moves.csv',format="csv", sep=",", inferSchema="true", header="true")
chess_eco_codes= spark.read.load('/content/chess/eco_codes.csv',format="csv", sep=",", inferSchema="true", header="true")

In [17]:
games_data=chess_games.toPandas()
moves_data=chess_moves.toPandas()
eco_codes_data=chess_eco_codes.toPandas()

In [18]:
chess_games.describe().toPandas()

,summary,game_id,game_order,event,site,date_played,round,white,black,result,...,white_title,black_title,winner,winner_elo,loser,loser_elo,winner_loser_elo_diff,eco,date_created,tournament_name
0,count,2938,2938,2938,2938,2938,2938,2938,2938,2938,...,53,53,2938,1105,2938,1102,2937,2937,2938,2938
1,mean,None,118.27161334240981,None,None,None,5.8580326752893095,None,None,None,...,None,None,None,2658.3257918552035,None,2613.50998185118,16.83690840994212,None,None,None
2,stddev,None,119.04640293289312,None,None,None,6.524552393766707,None,None,None,...,None,None,None,80.15889290248214,None,101.42116122323225,64.81388382505601,None,None,None
3,min,0028cc2e-5aaa-4645-a8e9-4d6b51c8f9db,1,Braingames WCC,Baguio City,1886.??.??,1.0,"Abulhul,T","Abulhul,T",0-1,...,GM,GM,"Acs,P",2271,"Abulhul,T",2076,-402,A00,2022-07-22T22:33:50+0000,FideChamp1993
4,max,ffdfe902-b93f-4b5d-826b-cab5c25a9d2f,418,World Championship 9th,USA/CAN,2021.12.10,48.0,"Zvjaginsev,V","Zvjaginsev,V",1/2-1/2,...,GM,GM,draw,2870,draw,2863,661,E99,2022-07-22T22:35:53+0000,WorldChamp2021


In [19]:
chess_games.show(25)

+--------------------+----------+--------+------------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+---------+----------+----------------+---------+---------------------+---+--------------------+---------------+
|             game_id|game_order|   event|        site|date_played|round|           white|           black| result|white_elo|black_elo|white_title|black_title|   winner|winner_elo|           loser|loser_elo|winner_loser_elo_diff|eco|        date_created|tournament_name|
+--------------------+----------+--------+------------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+---------+----------+----------------+---------+---------------------+---+--------------------+---------------+
|86e0b7f5-7b94-4ae...|         1|WCh 2021|   Dubai UAE| 2021.11.26|  1.0|Nepomniachtchi,I|       Carlsen,M|1/2-1/2|     2782|     2855|       null|       null|     draw|      null|       

In [20]:
chess_moves.describe().toPandas()

,summary,game_id,move_no,move_no_pair,player,notation,move,from_square,to_square,piece,...,fen_row8_black_count,fen_row1_black_value,fen_row2_black_value,fen_row3_black_value,fen_row4_black_value,fen_row5_black_value,fen_row6_black_value,fen_row7_black_value,fen_row8_black_value,move_sequence
0,count,252948,252948,252948,252948,252948,252948,252948,252948,252948,...,252948,252948,252948,252948,252948,252948,252948,252948,252948,252948
1,mean,None,50.6419698910448,25.572643389155083,None,None,None,None,None,None,...,3.041020288755001,0.20729557063111786,0.40236333159384535,0.4499778610623527,1.1286904818381644,2.2783655138605563,4.421968151556841,6.197554438066322,11.47177285449974,None
2,stddev,None,36.253902558263285,18.12682996315726,None,None,None,None,None,None,...,2.2895951387253755,1.1517193865998028,1.5596246963512985,1.5349871543680023,2.1103817481732094,2.7307261636138365,3.601168087389963,4.558657844279511,9.775526468404218,None
3,min,0028cc2e-5aaa-4645-a8e9-4d6b51c8f9db,1,1,"Abulhul,T",Ba1,a1a2,a1,a1,B,...,0,0,0,0,0,0,0,0,0,Nc3
4,max,ffdfe902-b93f-4b5d-826b-cab5c25a9d2f,291,146,"Zvjaginsev,V",hxg7+,h8h7,h8,h8,R,...,8,18,22,21,20,21,28,27,31,g3|g6|Bg2|Bg7|e4|c5|Ne2|Nc6|d3|Nf6|c4|O-O|O-O|...


In [21]:
chess_eco_codes.show(25)

+---+--------------------+--------------------+--------+---------+
|eco|            eco_name|         eco_example|eco_type|eco_group|
+---+--------------------+--------------------+--------+---------+
|A00|    Uncommon Opening|  1 g4, a3, h3, etc.|       A|    Flank|
|A01|Nimzovich-Larsen ...|                1 b3|       A|    Flank|
|A02|      Bird's Opening|                1 f4|       A|    Flank|
|A03|      Bird's Opening|             1 f4 d5|       A|    Flank|
|A04|        Reti Opening|               1 Nf3|       A|    Flank|
|A05|        Reti Opening|           1 Nf3 Nf6|       A|    Flank|
|A06|        Reti Opening|            1 Nf3 d5|       A|    Flank|
|A07|King's Indian Attack|       1 Nf3 d5 2 g3|       A|    Flank|
|A08|King's Indian Attack|1 Nf3 d5 2 g3 c5 ...|       A|    Flank|
|A09|        Reti Opening|       1 Nf3 d5 2 c4|       A|    Flank|
|A10|             English|                1 c4|       A|    Flank|
|A11|English, Caro-Kan...|             1 c4 c6|       A|    Fl

In [22]:
chess_moves.show(50)

+--------------------+-------+------------+----------------+--------+----+-----------+---------+-----+-----+--------------------+--------+-------------+--------------+----------------------+------------+------------------------+-----------+-----------+----------------+----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+----------------+----------------+------------------------+------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------

In [23]:
chess_games.count()

2938

In [24]:
chess_games.select("game_order").show()

+----------+
|game_order|
+----------+
|         1|
|         2|
|         3|
|         4|
|         5|
|         6|
|         7|
|         8|
|         9|
|        10|
|        11|
|         1|
|         2|
|         3|
|         4|
|         5|
|         6|
|         7|
|         8|
|         9|
+----------+
only showing top 20 rows



In [25]:
from pyspark.sql import functions as F

In [26]:
chess_games.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in chess_games.columns]).show()

+-------+----------+-----+----+-----------+-----+-----+-----+------+---------+---------+-----------+-----------+------+----------+-----+---------+---------------------+---+------------+---------------+
|game_id|game_order|event|site|date_played|round|white|black|result|white_elo|black_elo|white_title|black_title|winner|winner_elo|loser|loser_elo|winner_loser_elo_diff|eco|date_created|tournament_name|
+-------+----------+-----+----+-----------+-----+-----+-----+------+---------+---------+-----------+-----------+------+----------+-----+---------+---------------------+---+------------+---------------+
|      0|         0|    0|   0|          0|    0|    0|    0|     0|      587|      588|       2885|       2885|     0|      1833|    0|     1836|                    1|  1|           0|              0|
+-------+----------+-----+----+-----------+-----+-----+-----+------+---------+---------+-----------+-----------+------+----------+-----+---------+---------------------+---+------------+-------

In [27]:
#chess_history_games=chess_history_games.na.fill(value=0,subset=['white_elo',"black_elo","white_title","black_title","winner_elo","loser_elo","winner_loser_elo_diff","eco"])

In [28]:
spark

In [29]:
tournament_names=chess_games.select("tournament_name").distinct().collect()
tournament_names
tn=games_data.tournament_name.unique()

In [30]:
#from pyspark.sql.functions import desc

**1.List of Winners of Each World champions Trophy**

In [31]:
for i in tn:
   tournament_winner=chess_games.filter(chess_games.tournament_name==i)
   win=tournament_winner.filter(tournament_winner.winner!="draw").groupBy("tournament_name","winner").count().limit(1)
   win.show()

+---------------+---------+-----+
|tournament_name|   winner|count|
+---------------+---------+-----+
| WorldChamp2021|Carlsen,M|    4|
+---------------+---------+-----+

+---------------+------+-----+
|tournament_name|winner|count|
+---------------+------+-----+
+---------------+------+-----+

+---------------+---------------+-----+
|tournament_name|         winner|count|
+---------------+---------------+-----+
| WorldChamp2016|Carlsen, Magnus|    3|
+---------------+---------------+-----+

+---------------+---------------+-----+
|tournament_name|         winner|count|
+---------------+---------------+-----+
| WorldChamp2014|Carlsen, Magnus|    3|
+---------------+---------------+-----+

+---------------+---------------+-----+
|tournament_name|         winner|count|
+---------------+---------------+-----+
| WorldChamp2013|Carlsen, Magnus|    3|
+---------------+---------------+-----+

+---------------+---------+-----+
|tournament_name|   winner|count|
+---------------+---------+-----+

**2. List of Players with number of times they have won Tournament in descending order**

In [32]:
maxwin=chess_games.groupBy("winner").count().sort("count",ascending=False).withColumnRenamed("count","number_of_wins").withColumnRenamed("winner","player_name")
maxwin.show()

+--------------------+--------------+
|         player_name|number_of_wins|
+--------------------+--------------+
|                draw|          1515|
|     Lasker, Emanuel|            52|
|     Karpov, Anatoly|            47|
|  Botvinnik, Mikhail|            46|
|   Steinitz, William|            43|
| Alekhine, Alexander|            43|
|           Topalov,V|            32|
|      Kasparov, Gary|            31|
|             Anand,V|            30|
|    Smyslov, Vassily|            24|
|            Adams,Mi|            20|
|      Kasimdzhanov,R|            19|
|          Grischuk,A|            17|
|           Svidler,P|            16|
|      Adams, Michael|            16|
|            Shirov,A|            15|
|            Bareev,E|            15|
|             Dreev,A|            15|
|   Chigorin, Mikhail|            14|
|Khalifman, Alexander|            14|
+--------------------+--------------+
only showing top 20 rows



**3. Most and Least Popular eco move in world championship hist**

In [33]:
pop=chess_eco_codes.groupBy("eco_name").count().sort("count",ascending=False).withColumnRenamed("count","number_of_occurence")
pop.show()

+--------------------+-------------------+
|            eco_name|number_of_occurence|
+--------------------+-------------------+
|            Sicilian|                 15|
|             English|                 13|
|Queen's Gambit De...|                 11|
|   Queen's Pawn Game|                 10|
|       King's Indian|                 10|
|           Ruy Lopez|                  9|
|Queen's Gambit Ac...|                  7|
|English, Symmetrical|                  6|
|            Grunfeld|                  6|
|               Dutch|                  6|
|Queen's Gambit De...|                  6|
|      Queen's Indian|                  6|
|King's Gambit Acc...|                  5|
|   Sicilian, Najdorf|                  5|
|   Ruy Lopez, Closed|                  5|
|Queen's Gambit De...|                  5|
|        Reti Opening|                  4|
|         Two Knights|                  4|
|     Catalan, Closed|                  4|
|Nimzo-Indian, Cla...|                  4|
+----------

In [34]:
maxpop=chess_eco_codes.groupBy("eco_name").count().sort("count",ascending=False).withColumnRenamed("count","number_of_occurance").limit(1)
maxpop.show()

+--------+-------------------+
|eco_name|number_of_occurance|
+--------+-------------------+
|Sicilian|                 15|
+--------+-------------------+



In [35]:
maxpop=chess_eco_codes.groupBy("eco_name").count().sort("count",ascending=True).withColumnRenamed("count","number_of_occurance").limit(1)
maxpop.show()

+--------------------+-------------------+
|            eco_name|number_of_occurance|
+--------------------+-------------------+
|Sicilian, Acceler...|                  1|
+--------------------+-------------------+



**5. Longest and shortest game ever played in a world championship in terms of move.**

In [36]:
maxwin=chess_games.groupBy("winner").count().sort("count",ascending=False).withColumnRenamed("count","number_of_wins").withColumnRenamed("winner","player_name")
maxwin.show()

+-------------------+--------------+
|        player_name|number_of_wins|
+-------------------+--------------+
|               draw|          1515|
|    Lasker, Emanuel|            52|
|    Karpov, Anatoly|            47|
| Botvinnik, Mikhail|            46|
|  Steinitz, William|            43|
|Alekhine, Alexander|            43|
|          Topalov,V|            32|
|     Kasparov, Gary|            31|
|            Anand,V|            30|
|   Smyslov, Vassily|            24|
|           Adams,Mi|            20|
|     Kasimdzhanov,R|            19|
|         Grischuk,A|            17|
|          Svidler,P|            16|
|     Adams, Michael|            16|
|           Shirov,A|            15|
|           Bareev,E|            15|
|            Dreev,A|            15|
|  Chigorin, Mikhail|            14|
|          Euwe, Max|            14|
+-------------------+--------------+
only showing top 20 rows



In [37]:
maxmov=chess_moves.groupBy("move_no","move_no_pair").count().sort("move_no",ascending=False).limit(1)
maxmov.show()

+-------+------------+-----+
|move_no|move_no_pair|count|
+-------+------------+-----+
|    291|         146|    1|
+-------+------------+-----+



In [38]:
minmov=chess_moves.groupBy("move_no_pair","move_no").count().sort("move_no",ascending=True).limit(1)
minmov.show()

+------------+-------+-----+
|move_no_pair|move_no|count|
+------------+-------+-----+
|           1|      1| 2937|
+------------+-------+-----+



In [39]:
tournament_player=chess_moves.select("player").distinct().collect()
tournament_player

[Row(player='Hamdouchi, Hichem'),
 Row(player='Fridman, Daniel'),
 Row(player='Svidler, Peter'),
 Row(player='Alterman, Boris'),
 Row(player='Nepomniachtchi,I'),
 Row(player='Ivanchuk,V'),
 Row(player='Andersson, Ulf'),
 Row(player='Jussupow, Artur'),
 Row(player='Piket,Je'),
 Row(player='Van Wely, Loek'),
 Row(player='Onischuk,Al'),
 Row(player='Peng Xiaomin'),
 Row(player='Vladimirov,E'),
 Row(player='Kacheishvili,G'),
 Row(player='Luther,T'),
 Row(player='Ivanchuk, Vassily'),
 Row(player='Sakaev, Konstantin'),
 Row(player='Steinitz, William'),
 Row(player='Carlsen,M'),
 Row(player='Ivanov,AV'),
 Row(player='Istratescu, Andrei'),
 Row(player='Charbonneau,P'),
 Row(player='Tiviakov,S'),
 Row(player='Cvitan,O'),
 Row(player='Karpov, Anatoly'),
 Row(player='Gu Xiaobing'),
 Row(player='Miladinovic, Igor'),
 Row(player='Lerner, Konstantin Z'),
 Row(player='Kotsur, Pavel'),
 Row(player='Adams,Mi'),
 Row(player='Yu Shaoteng'),
 Row(player='Benjamin,Joe'),
 Row(player='Benjamin, Joel'),
 Row

In [40]:
tn_player=moves_data.player.unique()
tn_player

array(['Nepomniachtchi,I', 'Carlsen,M', 'Caruana, Fabiano',
       'Carlsen, Magnus', 'Karjakin, Sergey', 'Anand, Viswanathan',
       'Anand,V', 'Gelfand,B', 'Gelfand, Boris', 'Topalov,V', 'Kramnik,V',
       'Svidler,P', 'Morozevich,A', 'Aronian,L', 'Grischuk,A', 'Leko,P',
       'Adams,Mi', 'Polgar,Ju', 'Kasimdzhanov,R', 'Abulhul,T',
       'Asabri,Hu', 'Solomon,K', 'Ivanchuk,V', 'Arab,A', 'Kadhi,H',
       'Short,N', 'Malakhov,V', 'Haznedaroglu,K', 'Dableo,R',
       'Nisipeanu,LD', 'Sokolov,I', 'Simutowe,A', 'Tissir,M', 'Dreev,A',
       'Akopian,Vl', 'Gonzalez Garcia,J', 'Garcia Palermo,C',
       'Ye Jiangchuan', 'Mahjoob,M', 'Azmaiparashvili,Z', 'Bacrot,E',
       'Charbonneau,P', 'Johansen,D', 'Gurevich,M', 'Rublevsky,S',
       'Adly,A', 'Bartel,Mat', 'Radjabov,T', 'Aleksandrov,A',
       'El Gindy,E', 'Barsov,A', 'Beliavsky,A', 'Vallejo Pons,F',
       'Vasquez,R', 'Paragua,M', 'Bologan,V', 'Sakaev,K',
       'Mastrovasilis,D', 'Kritz,L', 'Sasikiran,K', 'Mamedyarov,S',
     

In [ ]:
from google.colab import drive
drive.mount('/content/drive')